In [1]:
import pandas as pd
import numpy as np 

In [2]:
 df = pd.read_csv("/content/drive/My Drive/Data Science Projects Data/fake-news.csv")

In [3]:
map = {1:"FAKE", 0:"REAL"}
df["label"] = df["label"].map(map)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  object
dtypes: int64(1), object(4)
memory usage: 812.6+ KB


In [5]:
df.isna().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [6]:
df.drop("author", axis=1, inplace=True)

In [7]:
df.drop(["title", "id"], axis=1, inplace=True)

In [8]:
df.dropna(inplace=True)

In [9]:
df.isna().sum()

text     0
label    0
dtype: int64

In [10]:
df["label"].value_counts()

REAL    10387
FAKE    10374
Name: label, dtype: int64

In [11]:
blank = []

for ind,txt,lb in df.itertuples():
    if type(txt)==str:
        if txt.isspace():
            blank.append(ind)

In [12]:
blank

[82,
 169,
 295,
 470,
 592,
 871,
 901,
 936,
 950,
 1591,
 2163,
 2228,
 2382,
 2684,
 3319,
 3322,
 3957,
 4275,
 4398,
 4718,
 4902,
 5012,
 5101,
 5563,
 5709,
 6119,
 6220,
 6443,
 6646,
 6976,
 7980,
 8340,
 8625,
 8802,
 9035,
 9571,
 9663,
 9854,
 10071,
 10153,
 10465,
 10627,
 11349,
 11373,
 11718,
 12129,
 12145,
 12374,
 12414,
 13008,
 13453,
 13680,
 13872,
 15102,
 15255,
 15299,
 15959,
 16347,
 16561,
 16715,
 17351,
 17417,
 17530,
 17794,
 17813,
 17819,
 17960,
 18675,
 18998,
 19672,
 19850,
 19857,
 19929,
 20141,
 20242,
 20264,
 20513]

In [13]:
df.drop(blank, axis=0, inplace=True)

In [14]:
df.reset_index(inplace=True)

In [15]:
df.drop("index", axis=1, inplace=True)

In [16]:
df.head()

,text,label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,FAKE
1,Ever get the feeling your life circles the rou...,REAL
2,"Why the Truth Might Get You Fired October 29, ...",FAKE
3,Videos 15 Civilians Killed In Single US Airstr...,FAKE
4,Print \nAn Iranian woman has been sentenced to...,FAKE


In [17]:
df["label"].value_counts()

REAL    10387
FAKE    10297
Name: label, dtype: int64

In [102]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
import string

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [21]:
corpus = []
ps = PorterStemmer()

for i in range(len(df)):
    
    nopunc = [char for char in df["text"][i] if char not in string.punctuation]
    
    word = "".join(nopunc)
    
    word = word.lower()
    
    word = word.split()
    
    words = [words for words in word if words not in set(stopwords.words("english"))]
    
    word = [ps.stem(word) for word in words]
    
    msg = " ".join(word)
    
    corpus.append(msg)

In [72]:
tfidf = TfidfVectorizer()
X = tfidf.fit_transform(corpus)

In [103]:
y = pd.get_dummies(df["label"])
y = y.iloc[:,1].values

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# from sklearn.l import MultinomialNB
from sklearn.linear_model import PassiveAggressiveClassifier
model = PassiveAggressiveClassifier()
model.fit(X_train, y_train)

import pickle
#creating pickle file for model
pickle.dump(model, open("fake_news-model.pkl","wb"))

#creating pickle file for vector
pickle.dump(cv, open("tfidf_news-vector.pkl", "wb"))

In [104]:
model.score(X_test, y_test)

0.9637418419144308

In [26]:
df.head()

,text,label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,FAKE
1,Ever get the feeling your life circles the rou...,REAL
2,"Why the Truth Might Get You Fired October 29, ...",FAKE
3,Videos 15 Civilians Killed In Single US Airstr...,FAKE
4,Print \nAn Iranian woman has been sentenced to...,FAKE


In [99]:
txt = df["text"].iloc[500]

In [100]:
txt = tfidf.transform([txt])

In [101]:
model.predict(txt)

array([1], dtype=uint8)

In [98]:
y[500]

1